# Table of Contents

01. Import Libraries
02. Import Data
03. Task 4.9 Part 1

# 01. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Import Data

In [2]:
path = r'C:\Users\Cathy\2022 Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived_grouped(3).pkl'))

In [4]:
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

# 03. Task 4.9 Part 1

In [5]:
customers.shape

(206209, 10)

In [6]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
#Step 4: data wrangling. Drop First Name, Surname, gender, and date_joined columns and create new dataframe.
df_cust = customers.drop(columns = ['First Name','Surnam','Gender','date_joined'])

In [6]:
df_cust.head()

,user_id,STATE,Age,n_dependants,fam_status,income
0,26711,Missouri,48,3,married,165665
1,33890,New Mexico,36,0,single,59285
2,65803,Idaho,35,2,married,99568
3,125935,Iowa,40,0,single,42049
4,130797,Maryland,26,1,married,40374


In [7]:
#Step 4 rename columns for consistency
df_cust.rename(columns = {'STATE' : 'state'}, inplace = True)

In [8]:
#Step 4 rename columns for consistency
df_cust.rename(columns = {'Age' : 'age'}, inplace = True)

In [9]:
#Step 4 rename columns for consistency
df_cust.rename(columns = {'n_dependants' : '#_dependants'}, inplace = True)

In [10]:
df_cust.head()

,user_id,state,age,#_dependants,fam_status,income
0,26711,Missouri,48,3,married,165665
1,33890,New Mexico,36,0,single,59285
2,65803,Idaho,35,2,married,99568
3,125935,Iowa,40,0,single,42049
4,130797,Maryland,26,1,married,40374


In [11]:
df_cust.value_counts(dropna = False)

user_id  state          age  #_dependants  fam_status                        income
1        Alabama        31   3             married                           40423     1
137478   Maryland       19   2             living with parents and siblings  28625     1
137468   Nevada         31   1             married                           42533     1
137469   New Hampshire  45   1             married                           125718    1
137470   New Jersey     19   3             living with parents and siblings  79969     1
                                                                                      ..
68740    Tennessee      48   3             married                           134149    1
68741    Texas          68   0             divorced/widowed                  31977     1
68742    Utah           33   0             single                            58471     1
68743    Vermont        70   1             married                           80221     1
206209   Iowa           74

In [12]:
df_cust.dtypes

user_id          int64
state           object
age              int64
#_dependants     int64
fam_status      object
income           int64
dtype: object

In [13]:
#Step 5 change data types to prepare for merge
df_cust['user_id']=df_cust['user_id'].astype('int32')
df_cust['age'] = df_cust['age'].astype('int8')
df_cust['#_dependants']=df_cust['#_dependants'].astype('int8')
df_cust['income']=df_cust['income'].astype('int32')

In [14]:
df_cust.dtypes

user_id          int32
state           object
age               int8
#_dependants      int8
fam_status      object
income           int32
dtype: object

In [15]:
#Step 5 check for mixed-type data
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

In [16]:
#Step 5 check for missing values
df_cust.isnull().sum()

user_id         0
state           0
age             0
#_dependants    0
fam_status      0
income          0
dtype: int64

In [17]:
#Step 5 check for duplicates
df_dups = df_cust[df_cust.duplicated()]

In [18]:
df_dups

,user_id,state,age,#_dependants,fam_status,income


In [19]:
#check user_id key column data types for both sets are int32
ords_prods_merge.dtypes

order_id                     int32
user_id                      int32
order_number                  int8
orders_day_of_week            int8
order_hour_of_day             int8
days_since_prior_order     float16
product_id                   int32
add_to_cart_order            int32
reordered                     int8
product_name                object
aisle_id                     int32
department_id                 int8
prices                     float64
_merge                    category
price_range_loc             object
busiest_days                object
busiest_period_of_day       object
max_order                     int8
loyalty_flag                object
Spending_flag               object
order_frequency            float16
frequency_flag              object
dtype: object

In [22]:
#Step 6 merge data frames (default inner join)
ords_prods_all = ords_prods_merge.merge(df_cust, on = 'user_id', indicator = 'exists')

In [23]:
ords_prods_all.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,loyalty_flag,Spending_flag,order_frequency,frequency_flag,state,age,#_dependants,fam_status,income,exists
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,New customer,Low Spender,20.5,Non-frequent customer,Alabama,31,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,New customer,Low Spender,20.5,Non-frequent customer,Alabama,31,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,New customer,Low Spender,20.5,Non-frequent customer,Alabama,31,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,New customer,Low Spender,20.5,Non-frequent customer,Alabama,31,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,New customer,Low Spender,20.5,Non-frequent customer,Alabama,31,3,married,40423,both


In [25]:
ords_prods_all.shape

(32434212, 28)

In [27]:
#find "max order <" for report
ords_prods_merge['max_order'].describe()

count    3.243421e+07
mean     3.305217e+01
std      2.515449e+01
min      3.000000e+00
25%      1.300000e+01
50%      2.600000e+01
75%      4.700000e+01
max      9.900000e+01
Name: max_order, dtype: float64

In [28]:
#export
ords_prods_all.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))